# Optimizacion Modelo Cosecha Uvas 2

In [1]:
from gurobipy import *
import numpy as np
import pandas as pd
import math as mt
import random
import scipy.stats as st
import matplotlib.pyplot as plt
import statsmodels.api as ss
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from statsmodels.graphics.factorplots import interaction_plot
from  statsmodels.stats.multicomp import pairwise_tukeyhsd


Bad key text.latex.unicode in file C:\Users\LabCivil1-Pc1\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 112 ('text.latex.unicode : False # use "ucs" and "inputenc" LaTeX packages for handling')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key savefig.frameon in file C:\Users\LabCivil1-Pc1\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 423 ('savefig.frameon : True')
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.3.4/matplotlibrc.template
or from the matplotlib source distribution

Bad key pgf.debug in file C:\Users\LabCivil1-Pc1\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle, line 444 ('pgf.debug           : False')
You probably need to get an updated matplotlibrc file from
https:/

# Metaheuristica Recocido Simulado Cosecha Uvas

### Definiendo Funciones

##### Parametros

In [2]:
def parametros(J,T,C):
    np.random.seed(1)
    #J=1
    #T=10
    bloques=[j+1 for j in range(J)]
    periodos=[t+1 for t in range(T)]
    #C=0.3
    C=C
    return bloques, periodos, C

##### Funcion Calidad

In [3]:
def Qt(t,a,b):  #Costo por perdida de calidad de la uva
        Qt=max(-a*(t-b)**2+1,0)
        return Qt

##### Conjuntos

In [4]:
bloques,periodos,C=parametros(J=1,T=10,C=0.3)

##### Modelo de Optimizacion

In [5]:
def optimizacion(bloques,periodos,a_q,b_q,C):
    m=Model("Optimizacion Cosecha Uvas2")
    m.setParam('LogToConsole', 0)
    x=m.addVars(bloques, periodos, lb=0, ub=1, vtype=GRB.CONTINUOUS,name="x")
    obj=quicksum(x[j,t]*Qt(t,a_q,b_q) for j in bloques for t in periodos)
    m.setObjective(obj, sense=GRB.MAXIMIZE)
    m.addConstrs(quicksum(x[j,t] for j in bloques)<=C for t in periodos)
    m.addConstrs(quicksum(x[j,t] for t in periodos)==1 for j in bloques)
    m.update()
    m.optimize()
    objetive=m.objVal
    X={(j,t):round(x[j,t].x) for j in bloques for t in periodos}
    #X={(j,t):0.3 for j in bloques for t in periodos}
    L={(j,t):0 for j in bloques for t in periodos}
    U={(j,t):C for j in bloques for t in periodos}
    
    return objetive,X,L,U

In [6]:
a_q=1/10
b_q=5
O,X,L,U=optimizacion(bloques=bloques,periodos=periodos,a_q=a_q,b_q=b_q,C=C)

Academic license - for non-commercial use only - expires 2022-01-12
Using license file C:\Users\LabCivil1-Pc1\gurobi.lic


##### Funcion de Penalidad

In [7]:
def I(X,L,U):
    sum1=0
    for t in periodos:
        sum1+=max(0,sum(X[j,t] for j in bloques)-C)
    
    sum2=0
    for j in bloques:
        sum2+=max(0,abs(sum(X[j,t] for t in periodos)-1))
    
    sum3=0
    for j in bloques:
        for t in periodos:
            sum3+=(max(0,L[j,t]-X[j,t])+max(0,X[j,t]-U[j,t]))
    
    I=sum1+sum2+sum3
    
    return I

##### Funcion Objetivo

In [8]:
def Z(X):
    z=sum(X[j,t]*Qt(t,a_q,b_q) for j in bloques for t in periodos)
    return z

##### Metaheuristica Recocido Simulado

In [9]:
def SA(iteraciones,T_o,r,a_o,I_o,Z_o,a,n,k,k_,E,Zs,Xs,X):
    for i in range(iteraciones):
        l=k*random.randrange(-1,2,2)
        z=np.random.randint(1,len(periodos))
        #print(z)
        #print(l)
        N=dict(X)
        N[1,z]=min(max(N[1,z]+l,L[1,z]),U[1,z])
        Xt=dict(N)
        #print(Xt)
        It=I(Xt,L,U)
        #print(It)
        if It<I_o:
            I_o=It
        if Xt not in E:
            E.append(Xt)
            if It>0:
                #E.append(Xt)
                X=dict(Xt)
            else:
                Zt=Z(Xt)
                if Zt>=Z_o:
                    E.append(Xt)
                    Xs.append(Xt)
                    X=dict(Xt)
                    Z_o=Zt
                    Zs[i,l]=Zt
                    a+=1
                else:
                    if np.random.random()<=mt.e**(-abs(Z_o-Zt)/T_o):
                        E.append(Xt)
                        Xs.append(Xt)
                        X=dict(Xt)
                        Z_o=Zt
                        Zs[i,l]=Zt
                        a+=1
        if a==a_o:
            a=0
            n+=1
            T_o=float(r*T_o)
    return n,Z_o,I_o,T_o

##### Prueba Metaheuristica

In [10]:
T_o=0.75*O
r=0.5
a_o=2
iteraciones=5000
I_o=99999
Z_o=-99999
a=0
n=0
k=0.1
k_=list(np.linspace(-k,k,2))
E=[]
E.append(X)
Zs={(i,l):0 for i in range(iteraciones) for l in k_}
Xs=[]
n,Z_o,I_o,T_o=SA(iteraciones=iteraciones,T_o=T_o,r=r,a_o=a_o,I_o=I_o,Z_o=Z_o,a=a,n=n,k=k,k_=k_,E=E,Zs=Zs,Xs=Xs,X=X)

In [11]:
n,Z_o,I_o,T_o,Xs

(8,
 0.68,
 0,
 0.00263671875,
 [{(1, 1): 0.1,
   (1, 2): 0.2,
   (1, 3): 0.0,
   (1, 4): 0,
   (1, 5): 0,
   (1, 6): 0.2,
   (1, 7): 0.0,
   (1, 8): 0.3,
   (1, 9): 0.2,
   (1, 10): 0},
  {(1, 1): 0.0,
   (1, 2): 0.3,
   (1, 3): 0.1,
   (1, 4): 0.3,
   (1, 5): 0.0,
   (1, 6): 0.09999999999999998,
   (1, 7): 0.2,
   (1, 8): 0.0,
   (1, 9): 0.0,
   (1, 10): 0},
  {(1, 1): 0.0,
   (1, 2): 0.19999999999999998,
   (1, 3): 0.2,
   (1, 4): 0.3,
   (1, 5): 0.0,
   (1, 6): 0.09999999999999998,
   (1, 7): 0.2,
   (1, 8): 0.0,
   (1, 9): 0.0,
   (1, 10): 0},
  {(1, 1): 0.0,
   (1, 2): 0.09999999999999998,
   (1, 3): 0.3,
   (1, 4): 0.3,
   (1, 5): 0.0,
   (1, 6): 0.09999999999999998,
   (1, 7): 0.2,
   (1, 8): 0.0,
   (1, 9): 0.0,
   (1, 10): 0},
  {(1, 1): 0.0,
   (1, 2): 0.1,
   (1, 3): 0,
   (1, 4): 0.19999999999999998,
   (1, 5): 0.1,
   (1, 6): 0.1,
   (1, 7): 0,
   (1, 8): 0.19999999999999998,
   (1, 9): 0.3,
   (1, 10): 0},
  {(1, 1): 0.0,
   (1, 2): 0.1,
   (1, 3): 0,
   (1, 4): 0.3,
   

# Generacion Datos Experimento (n=5)

#### Diferentes a_q

#### Diferentes T_o

#### Diferentes r

#### Diferentes k

#### Diferentes iteraciones

# Experimento1 2k

In [ ]:
#Combinaciones de parametros a_qs, T_os, r_s, k_s, iteraciones

rows=[]
a_qs=[1,20]
T_os=[0.1,0.9]
r_s=[0.1,0.9]
k_s=[0.05,0.25]
iteraciones=[500,10000]
replicas=10
comb=[(q,j,y,e,g) for q in a_qs for j in T_os for y in r_s for e in k_s for g in iteraciones for s in range(replicas)]
random.shuffle(comb)

for (q,j,y,e,g) in comb:
    bloques,periodos,C=parametros(J=1,T=10,C=0.3)
    #a_q=1/10
    a_q=q
    b_q=5
    O,X,L,U=optimizacion(bloques=bloques,periodos=periodos,a_q=a_q,b_q=b_q,C=C)
    T_o=j*O #Temperatura modificada
    #r=0.5
    a_o=2
    #iteraciones=5000
    iteraciones=g
    I_o=999999
    Z_o=-99999
    a=0
    i=0
    #k=0.1
    k_=list(np.linspace(-e,e,2))
    E=[]
    E.append(X)
    Zs={(i,l):0 for i in range(iteraciones) for l in k_}
    Xs=[]
    n,Z_o,I_o,T_o=SA(iteraciones=iteraciones,T_o=T_o,r=y,a_o=a_o,I_o=I_o,Z_o=Z_o,a=a,n=n,k=e,k_=k_,E=E,Zs=Zs,Xs=Xs,X=X)
    #print(j,y,e,O,Z_o,n,I_o,T_o) 
    row = {
        'aq':q,
        'temperatura':j*O,
        'reduccion':y,
        'paso':e,
        'iteraciones':g,
        'objective_base':O,
        'objective_sa':Z_o,
        'gap':abs(O-Z_o)/O if Z_o!=-99999 else 1,
        'n':n,
        'I_o':I_o,
        'T_of':T_o,
    }
    rows.append(row)
df_datos = pd.DataFrame(rows)

In [ ]:
df_datos.to_excel('DatosExperimento.xlsx', index=0)

In [ ]:
df_datos

# Experimento2 2k

In [12]:
#Combinaciones de parametros T_os, r_s, k_s, iteraciones

rows=[]
T_os=[0.1,0.9]
r_s=[0.1,0.9]
k_s=[0.05,0.25]
iteraciones=[500,10000]
replicas=10
comb=[(j,y,e,g) for j in T_os for y in r_s for e in k_s for g in iteraciones for s in range(replicas)]
random.shuffle(comb)

for (j,y,e,g) in comb:
    bloques,periodos,C=parametros(J=1,T=10,C=0.3)
    a_q=1/10
    b_q=5
    O,X,L,U=optimizacion(bloques=bloques,periodos=periodos,a_q=a_q,b_q=b_q,C=C)
    T_o=j*O #Temperatura modificada
    #r=0.5
    a_o=2
    #iteraciones=5000
    iteraciones=g
    I_o=999999
    Z_o=-99999
    a=0
    i=0
    #k=0.1
    k_=list(np.linspace(-e,e,2))
    E=[]
    E.append(X)
    Zs={(i,l):0 for i in range(iteraciones) for l in k_}
    Xs=[]
    n,Z_o,I_o,T_o=SA(iteraciones=iteraciones,T_o=T_o,r=y,a_o=a_o,I_o=I_o,Z_o=Z_o,a=a,n=n,k=e,k_=k_,E=E,Zs=Zs,Xs=Xs,X=X)
    #print(j,y,e,O,Z_o,n,I_o,T_o) 
    row = {
        'temperatura':j*O,
        'reduccion':y,
        'paso':e,
        'iteraciones':g,
        'objective_base':O,
        'objective_sa':Z_o,
        'gap':abs(O-Z_o)/O if Z_o!=-99999 else 1,
        'n':n,
        'I_o':I_o,
        'T_of':T_o,
    }
    rows.append(row)
df2_datos = pd.DataFrame(rows)

In [13]:
df2_datos.to_excel('DatosExperimento2.xlsx', index=0)

# Experimento22 2k

In [17]:
#Combinaciones de parametros r_s, k_s, iteraciones

rows=[]
r_s=[0.1,0.9]
k_s=[0.05,0.25]
iteraciones=[500,10000]
replicas=10
comb=[(y,e,g) for y in r_s for e in k_s for g in iteraciones for s in range(replicas)]
random.shuffle(comb)

for (y,e,g) in comb:
    bloques,periodos,C=parametros(J=1,T=10,C=0.3)
    a_q=1/10
    b_q=5
    O,X,L,U=optimizacion(bloques=bloques,periodos=periodos,a_q=a_q,b_q=b_q,C=C)
    T_o=0.75*O #Temperatura modificada
    #r=0.5
    a_o=2
    #iteraciones=5000
    iteraciones=g
    I_o=999999
    Z_o=-99999
    a=0
    i=0
    #k=0.1
    k_=list(np.linspace(-e,e,2))
    E=[]
    E.append(X)
    Zs={(i,l):0 for i in range(iteraciones) for l in k_}
    Xs=[]
    n,Z_o,I_o,T_o=SA(iteraciones=iteraciones,T_o=T_o,r=y,a_o=a_o,I_o=I_o,Z_o=Z_o,a=a,n=n,k=e,k_=k_,E=E,Zs=Zs,Xs=Xs,X=X)
    #print(j,y,e,O,Z_o,n,I_o,T_o) 
    row = {
        'reduccion':y,
        'paso':e,
        'iteraciones':g,
        'objective_base':O,
        'objective_sa':Z_o,
        'gap':abs(O-Z_o)/O if Z_o!=-99999 else 1,
        'n':n,
        'I_o':I_o,
        'T_of':T_o,
    }
    rows.append(row)
df22_datos = pd.DataFrame(rows)

In [18]:
df22_datos.to_excel('DatosExperimento22.xlsx', index=0)

# Experimento3 2k

In [14]:
#Combinaciones de parametros r_s, k_s, iteraciones

rows=[]
r_s=[0.1,0.3,0.5,0.7,0.9]
k_s=[0.05,0.1,0.15,0.2,0.25]
iteraciones=[500,2500,5000,7000,10000]
replicas=10
comb=[(y,e,g) for y in r_s for e in k_s for g in iteraciones for s in range(replicas)]
random.shuffle(comb)

for (y,e,g) in comb:
    bloques,periodos,C=parametros(J=1,T=10,C=0.3)
    a_q=1/10
    b_q=5
    O,X,L,U=optimizacion(bloques=bloques,periodos=periodos,a_q=a_q,b_q=b_q,C=C)
    T_o=0.75*O
    #r=0.5
    a_o=2
    #iteraciones=5000
    iteraciones=g
    I_o=999999
    Z_o=-99999
    a=0
    i=0
    #k=0.1
    k_=list(np.linspace(-e,e,2))
    E=[]
    E.append(X)
    Zs={(i,l):0 for i in range(iteraciones) for l in k_}
    Xs=[]
    n,Z_o,I_o,T_o=SA(iteraciones=iteraciones,T_o=T_o,r=y,a_o=a_o,I_o=I_o,Z_o=Z_o,a=a,n=n,k=e,k_=k_,E=E,Zs=Zs,Xs=Xs,X=X)
    #print(j,y,e,O,Z_o,n,I_o,T_o) 
    row = {
        'reduccion':y,
        'paso':e,
        'iteraciones':g,
        'objective_base':O,
        'objective_sa':Z_o,
        'gap':abs(O-Z_o)/O if Z_o!=-99999 else 1,
        'n':n,
        'I_o':I_o,
        'T_of':T_o,
    }
    rows.append(row)
df3_datos = pd.DataFrame(rows)

In [15]:
df3_datos.to_excel('DatosExperimento3.xlsx', index=0)